# TCAD Preprocessing

TCAD provides an export files with the data in their database as zip files. These must be downloaded and parsed.

In [1]:
%load_ext autoreload
%autoreload 2

import tcad.tparser as tcp

## Download TCAD Files

The TCAD Appraisal Export can be found publically on the public information section of the website under resources: https://traviscad.org/publicinformation

In this notebook, we use the "2023 Certified Export (July)" file. Download and extract contents of the zip file to the raw folder.

Direct download link: https://traviscad.org/wp-content/largefiles/2023%20Certified%20Appraisal%20Export%20Supp%200_07232022.zip

In [13]:
# Deprecated
# import tcad.downloader as tcd

# tcd.download()
# tcd.extract()

# Provide the directory where you extracted the files here.
RAW_TCAD_DIR = 'data/raw/2023_Certified_Appraisal_Export_Supp_0_07232022'

# Where you want the files from this notebook to be saved.
PROCESSED_TCAD_DIR = 'data/processed/TCAD'

## Layout File

Also in the public information section, there is the "Export Layouts" file. This files serves the techical documentation or data dictionary for the database export since the TCAD export consists of a number of text files that are in the fixed width format (not comma or tab). The layout file tells you the start and end position for the fields in each row. This file is automatically downloaded for parsing purposes.

Direct download link: https://traviscad.org/wp-content/largefiles/Legacy8.0.25-Export-Layouts-07242023.zip

In [14]:
# tcp.download_layout()

## Improvement Info

Improvement info contains overview information for each property like the type of building as well as the total value of buildings on the site. There is one row per property. This function is used here for demonstration purposes. It's is already called from within parse_improvement_info and does not need to be called separately.

In [47]:
tcp.load_improvement_info_layout()

,Field Name,Datatype,Start,End,Length,Description,col_spec
0,prop_id,int(12),1,12,12,Property ID,"(0, 12)"
1,prop_val_yr,numeric(4),13,16,4,Year,"(12, 16)"
2,imprv_id,int(12),17,28,12,Improvement ID,"(16, 28)"
3,imprv_type_cd,varchar(10)`,29,38,10,Improvement Type,"(28, 38)"
4,imprv_type_desc,varchar(25),39,63,25,Improvement Description,"(38, 63)"
5,imprv_state_cd,varchar(5),64,68,5,State Code,"(63, 68)"
6,imprv_homesite,varchar(1),69,69,1,"""Y"" indicates Homesite improvement, ""N"" non Ho...","(68, 69)"
7,imprv_val,numeric(14),70,83,14,Improvement Value,"(69, 83)"
8,imprv_homesite_pct,numeric(15),84,98,15,Homesite Apportionment Percentage,"(83, 98)"
9,omitted,varchar(1),99,99,1,"Omitted Improvement (""Y"" or ""N"")","(98, 99)"


In [72]:
tcp.parse_improvement_info(f'{RAW_TCAD_DIR}/IMP_INFO.txt',
                           export_file=f'{PROCESSED_TCAD_DIR}/IMP_INFO.parquet')

,prop_id,prop_val_yr,imprv_id,imprv_type_cd,imprv_type_desc,imprv_state_cd,imprv_homesite,imprv_val,imprv_homesite_pct,omitted,omitted_imprv_val
0,100008,2023,5225541,00,Detail Only,F1,N,12278,100.0,N,0
1,100008,2023,5225542,32,RESTAURANT,F1,N,176710,100.0,N,0
2,100012,2023,5225543,08,APARTMENT 100+,B1,N,38413766,100.0,N,0
3,100012,2023,5225544,24,COMMERCIAL SPACE CONDOS,F1,N,1432068,100.0,N,0
4,100012,2023,5225545,19,SPECIAL (NODEPR),B1,N,0,100.0,N,0
...,...,...,...,...,...,...,...,...,...,...,...
422189,977351,2023,5807305,11,MOHO SINGLE PP,M1,N,11648,100.0,N,0
422190,977368,2023,5807423,12,MOHO DOUBLE PP,M1,N,107388,100.0,N,0
422191,977370,2023,5807440,12,MOHO DOUBLE PP,M1,Y,129739,100.0,N,0
422192,977394,2023,5807584,11,MOHO SINGLE PP,M1,Y,38485,100.0,N,0


## Improvement Details

Improvement details tracks the changes to the property overtime in terms of its features (mainly the floors). There can be multiple rows per property (i.e. one row for each floor of a house and the year it was built and the floor area). In addition to floors, "HVAC" is also a possible entry of interest. However, this data is dependent on the accuracy of records.

For instance, the moontower apartment complex has all its floor area assigned one floor. With the super-coop building in austin, there are rows for floors 3-5, but floors 1 and 2 are combined into a row named "ADDL".

In [73]:
tcp.parse_improvement_details(f'{RAW_TCAD_DIR}/IMP_DET.txt',
                              export_file=f'{PROCESSED_TCAD_DIR}/IMP_DET.parquet')

,prop_id,prop_val_yr,imprv_id,imprv_det_id,Imprv_det_type_cd,Imprv_det_type_desc,Imprv_det_class_cd,yr_built,depreciation_yr,imprv_det_area,imprv_det_val
0,100008,2023,5225541,41038597,551,PAVED AREA,AA,1984,1984,17100,0
1,100008,2023,5225542,41038598,1ST,1st Floor,C,2013,2013,2986,161525
2,100008,2023,5225542,41038599,501,CANOPY,A,2013,2013,402,3036
3,100008,2023,5225542,41038600,611,TERRACE,CA,2013,2013,402,760
4,100008,2023,5225542,41038601,511,DECK,WA,2013,2013,850,1369
...,...,...,...,...,...,...,...,...,...,...,...
3150901,977351,2023,5807305,45612257,1ST,1st Floor,MH5,1995,1995,896,11648
3150902,977368,2023,5807423,45613519,1ST,1st Floor,MH8,2019,2019,1440,107388
3150903,977370,2023,5807440,45613646,1ST,1st Floor,MH8,2020,2020,1790,129739
3150904,977394,2023,5807584,45615062,1ST,1st Floor,MH6,2001,2001,1190,38485


## Improvement Features

Improvement features is another file that contains overall details about the house. Strangely, each field seems to have its own rows so there are multiple per property. Information of interest here may be the roof shape/material and foundation type.

In [50]:
tcp.load_improvement_features_layout()

,Field Name,Datatype,Start,End,Length,Description,col_spec,dtype
0,prop_id,int(12),1,12,12,Property ID,"(0, 12)",int64
1,prop_val_yr,numeric(4),13,16,4,Year,"(12, 16)",int16
2,imprv_id,int(12),17,28,12,Improvement ID,"(16, 28)",int64
3,imprv_det_id,int(12),29,40,12,Improvement Detail ID,"(28, 40)",int64
4,imprv_attr_id,int(12),41,52,12,Improvement Attribute ID,"(40, 52)",int64
5,imprv_attr_desc,varchar(25),53,77,25,Attribute Description,"(52, 77)",None
6,imprv_attr_cd,varchar(10),78,87,10,Attribute Code,"(77, 87)",None


In [74]:
tcp.parse_improvement_features(f'{RAW_TCAD_DIR}/IMP_ATR.txt',export_file=f'{PROCESSED_TCAD_DIR}/IMP_ATR.parquet')

,prop_id,prop_val_yr,imprv_id,imprv_det_id,imprv_attr_id,imprv_attr_desc,imprv_attr_cd
0,100008,2023,5225542,41038598,35044442,Floor Factor,1ST
1,100008,2023,5225542,41038598,35923380,Foundation,SLAB
2,100008,2023,5225542,41038598,36271876,Grade Factor,A
3,100008,2023,5225542,41038598,37645636,Shape Factor,I
4,100012,2023,5225543,41038605,35346999,Floor Factor,1ST
...,...,...,...,...,...,...,...
3362247,977341,2023,5807214,45611448,40098776,Shape Factor,R
3362248,977345,2023,5807244,45611722,40099051,Shape Factor,R
3362249,977346,2023,5807254,45611800,40099161,Floor Factor,1ST
3362250,977346,2023,5807254,45611800,40099162,Shape Factor,R


## Property Details

The property details files is where the bulk of the TCAD records data is stored. Data of interest here includes the property address, property value as well as other locational or tax related data. Due to how large and sparse the data was, many columns were dropped to improve performance. For instance, the addresses of the property owner, appraisal agent, insurances company as well as serveral various tax exemption information did not seem relevant. Of about 440 columns, 57 were kept.

In [51]:
tcp.load_property_layout()

,Field Name,Datatype,Start,End,Length,Description,col_spec,dtype
0,prop_id,int(12),1,12,12,Property ID,"(0, 12)",int64
1,prop_type_cd,char(5),13,17,5,Property Type Code:\nR = Real\nP = Business Pe...,"(12, 17)",None
2,prop_val_yr,numeric(5),18,22,5,Appraisal or Tax Year,"(17, 22)",int16
3,geo_id,char(50),547,596,50,Geographic ID,"(546, 596)",None
4,partial_owner,char(1),679,679,1,Partial Owner ('T' or 'F')\nThis indicates if ...,"(678, 679)",None
5,udi_group,int(12),680,691,12,This field indicates the UDI group that a part...,"(679, 691)",int64
6,situs_street_prefx,char(10),1040,1049,10,Situs/Location Street Prefix,"(1039, 1049)",None
7,situs_street,char(50),1050,1099,50,Situs/Location Street,"(1049, 1099)",None
8,situs_street_suffix,char(10),1100,1109,10,Situs/Location Street Suffix,"(1099, 1109)",None
9,situs_city,char(30),1110,1139,30,Situs/Location City,"(1109, 1139)",None


In [75]:
tcp.parse_property_details(f'{RAW_TCAD_DIR}/PROP.txt',export_file=f'{PROCESSED_TCAD_DIR}/PROP.parquet')

,prop_id,prop_type_cd,prop_val_yr,geo_id,partial_owner,udi_group,situs_street_prefx,situs_street,situs_street_suffix,situs_city,...,dataset_id,deed_num,exemption_percentage,sic_code,omitted_imprv_hstd_val,omitted_imprv_non_hstd_val,pp_late_interstate_allocation_val,appraised_val_reflecting_productivity_loss,assessed_val_reflecting_productivity_loss,late_correction_val
0,100008,R,2023,0100030105,F,0,S,LAMAR,BLVD,NaN,...,0,2014035621TR,0,NaN,0,0,0,0,0,0
1,100012,R,2023,0100030109,F,0,S,LAMAR,BLVD,NaN,...,0,2011172140TR,0,NaN,0,0,0,0,0,0
2,100015,R,2023,0100030112,F,0,S,LAMAR,BLVD,NaN,...,0,2001096152TR,0,NaN,0,0,0,0,0,0
3,100018,R,2023,0100030115,F,0,S,LAMAR,BLVD,NaN,...,0,2014043470TR,0,NaN,0,0,0,0,0,0
4,100020,R,2023,0100030201,F,0,S,LAMAR,BLVD,NaN,...,0,2007035338TR,0,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477141,977395,P,2023,NaN,F,0,NaN,KEVIN,LN,AUSTIN,...,0,NaN,0,NaN,0,0,0,0,0,0
477142,977396,P,2023,NaN,F,0,NaN,VARIOUS LOCATIONS,NaN,NaN,...,0,NaN,0,NaN,0,0,0,0,0,0
477143,977397,MH,2023,0259410216,F,0,NaN,KILLINGSWORTH,LN,PFLUGERVILLE,...,0,NaN,0,NaN,0,0,0,0,0,0
477144,977401,P,2023,NaN,F,0,NaN,PANTHER,LN,PFLUGERVILLE,...,0,NaN,0,NaN,0,0,0,0,0,0
